### embedding_dataset.pt 파일 생성하는 함수
동우님이 주신 것과 버전 호환성 문제가 생겨서 다시 만드는 파일을 임의로 추가했습니다.

In [1]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm

# 1. 데이터 로드
print("labeled.csv 파일 로딩 중...")
try:
    labeled_df = pd.read_csv('labeled.csv', encoding='utf-8')
except UnicodeDecodeError:
    # 인코딩 문제가 있는 경우 다른 인코딩 시도
    labeled_df = pd.read_csv('labeled.csv', encoding='cp949')

print(f"총 {len(labeled_df)}개의 데이터 로드됨")
print("데이터 구조:")
print(labeled_df.head())
print(f"컬럼: {labeled_df.columns.tolist()}")

# 2. 모델 로드
print("\nlegal-kr-sbert-contrastive 모델 로딩 중...")
semantic_dir = "../model/legal-kr-sbert-contrastive"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
semantic_model = SentenceTransformer(semantic_dir).to(device)
print(f"모델이 {device}에 로드됨")

# 3. 텍스트 전처리 (필요시)
texts = labeled_df['text'].fillna('').astype(str).tolist()
print(f"\n임베딩할 텍스트 개수: {len(texts)}")

# 4. 배치 단위로 임베딩 생성 (메모리 효율성)
batch_size = 32  # GPU 메모리에 따라 조정
embeddings_list = []

print("텍스트 임베딩 생성 중...")
for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = semantic_model.encode(
        batch_texts,
        convert_to_tensor=True,
        device=device,
        show_progress_bar=False
    )
    embeddings_list.append(batch_embeddings.cpu())

# 5. 모든 임베딩 합치기
embeddings = torch.cat(embeddings_list, dim=0)
print(f"임베딩 생성 완료: {embeddings.shape}")

# 6. 검색을 위한 데이터셋 구성
dataset = {
    'texts': texts,  # 원본 텍스트
    'embeddings': embeddings,  # 임베딩 벡터
    'metadata': {
        'filenames': labeled_df['filename'].tolist(),
        'labels': labeled_df['label'].tolist(),
        'basis': labeled_df['basis'].tolist()
    }
}

# 7. .pt 파일로 저장
print("\nembedding_dataset.pt 파일 저장 중...")
torch.save(dataset, 'embedding_dataset.pt')
print("저장 완료!")

# 8. 저장된 파일 검증
print("\n저장된 파일 검증...")
loaded_dataset = torch.load('embedding_dataset.pt', map_location='cpu')
print(f"텍스트 개수: {len(loaded_dataset['texts'])}")
print(f"임베딩 크기: {loaded_dataset['embeddings'].shape}")
print(f"메타데이터 키: {loaded_dataset['metadata'].keys()}")
print("검증 완료!")


c:\Users\jhg-pc-02\Documents\TextMining\TextMining\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


labeled.csv 파일 로딩 중...
총 8000개의 데이터 로드됨
데이터 구조:
               filename label  \
0  001_개인정보취급방침_가공.json    유리   
1   001_결혼정보서비스_가공.json    유리   
2        001_보증_가공.json    유리   
3      001_사이버몰_가공.json    유리   
4      001_상해보험_가공.json    유리   

                                                text basis  
0  제2조(개인정보의 처리 및 보유기간) \n① 협회는 법령에 따른 개인정보 보유․이용...   NaN  
1  제3조 (회원가입)\n① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제공...   NaN  
2  제2조(보증금액)\n ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부...   NaN  
3  제3조 (약관 등의 명시와 설명 및 개정)\n① 몰은 이 약관의 내용과 상호 및 대...   NaN  
4  제4조(보험금 지급에 관한 세부규정)\n② 제3조(보험금의 지급사유) 제2호에서 장...   NaN  
컬럼: ['filename', 'label', 'text', 'basis']

legal-kr-sbert-contrastive 모델 로딩 중...


c:\Users\jhg-pc-02\Documents\TextMining\TextMining\.venv\lib\site-packages\torch\cuda\__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


모델이 cpu에 로드됨

임베딩할 텍스트 개수: 8000
텍스트 임베딩 생성 중...


100%|██████████| 250/250 [17:53<00:00,  4.29s/it]
C:\Users\jhg-pc-02\AppData\Local\Temp\ipykernel_16284\66844664.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_

임베딩 생성 완료: torch.Size([8000, 384])

embedding_dataset.pt 파일 저장 중...
저장 완료!

저장된 파일 검증...
텍스트 개수: 8000
임베딩 크기: torch.Size([8000, 384])
메타데이터 키: dict_keys(['filenames', 'labels', 'basis'])
검증 완료!
